In [3]:
BRANCH='main'

In [4]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

'\nYou can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.\n\nInstructions for setting up Colab are as follows:\n1. Open a new Python 3 notebook.\n2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)\n3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)\n4. Run this cell to set up dependencies.\n'

# Introduction

In this notebook we demonstrate how to use p-tunining and prompt tuning within NeMo-Megatron. Both methods are parameter efficient alternatives to fine-tuning pretrained language models. Our NeMo implementation makes it possible to use one pretrained GPT model on many downstream tasks without needing to tune the model's full set of parameters. It also allows for adding new tasks to your model without overwriting or disrupting previous tasks on which the model has already been p-tuned/prompt-tuned. Because the original model parameters are frozen and never altered by either method, p-tuning/prompt-tuning also avoid cartographic forgetting issues often encountered when fine-tuning models. 

- Our prompt tuning implementation is based off Lester et. al’s EMNLP 2021 paper [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)

- Our p-tuning implementation is based off Liu et al's paper [GPT Understands, Too](https://arxiv.org/abs/2103.10385).

- Usage examples and API documentation can be found in [our user docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/megatron.html#prompt-tuning). 

<img src="images/prompt_learning_forward_pass.png" alt="Prompt Learning Forward Pass"/>


# Techincal Overview
Instead of selecting discrete text prompts in a manual or automated fashion, prompt-tuning and p-tuning utilize virtual prompt tokens that can be optimized via gradient decent. The only difference between prompt-tuning and p-tuning within NeMo-Megatron is the architecture used to tune the soft prompt tokens during training. 

### Terminology
We will be using the terms `continuous`, `soft`, and `virtual` interchangeably to refer to tokens inserted into the model prompt that have *no* concrete mapping to strings or characters within the model’s vocabulary. These virtual tokens exist in contrast to `discrete`, `hard`, or `real` tokens that *do* make up the model’s vocabulary.  Virtual tokens are purely 1D vectors with dimensionality equal to that of each real token embedding matching the `hidden_size` hyperparameter. 

In training and inference, continuous token embeddings are inserted amoung discrete token embeddings according to a template you provide in the model's config. We will show how to do this in the code below. 


### Prompt-Tuning
When prompt-tuning a pretrained GPT model, soft prompt embeddings are initialized as a 2D matrix of size `total_virtual_tokens X hidden_size`. Each task the model is prompt-tuned to perform has its own 2D embedding matrix associated with it. Tasks do not share any parameters during traning or inference. All GPT model parameters are frozen and only the embedding parameters for each task are updated during training. 

In prompt tuning you can specify how the embeddings are initialized for each task. You can either 

1. Initialize embedding parameters according to some random distribution
2. Initialize embedding parameters from existing vocabulary embeddings (recommended)

If you choose to initialize virtual token embeddings from existing embedding weights, you can provide the string of words you want to use for initialization in the model's config. This string will be tokenized and tiled or truncated to match the specified number of virtual tokens you would like to use (`total_virtual_tokens`). Vocab embeddings are copied and used to initialize the soft prompt embedding matrix for each task. The vocab embeddings themselves are not updated or changed during prompt tuning. 


### P-Tuning
When p-tuning, an LSTM model is used to predict virtual token embeddings. LSTM parameters are randomly initialized at the start of p-tuning. All GPT model parameters are frozen, and only the LSTM weights are updated at each training step. LSTM parameters are shared between all tasks that are p-tuned at the same time, but the LSTM model outputs unique virtual token embeddings for each task. The virtual tokens predicted by the LSTM are inserted among the discrete token input in the exact same manner as with prompt-tuning. You still specify the number of virtual tokens you want to use by setting `total_virtual_tokens` and each virtual token embedding is still a 1D vector of size `hidden_size`.


# The Best of Both
A single pretrained GPT model can use both p-tuning and prompt-tuning. While you must decide to use either p-tuning or prompt-tuning for each task you want your model to perform, you can p-tune your model on a set of tasks *A*, then prompt tune your same model on a different set of tasks *B* and run inference on tasks from both *A* and *B* at the same time. During prompt-tuning or p-tuning, tasks tuned at the same time must use the same number of virtual tokens. During inference, tasks using differing amounts of virtual tokens can be run at the same time. 

P-tuning usually requires fewer virtual tokens per task to achieve good results but uses a higher number of parameters (~ 6% original model params) compared to prompt-tuning (~ .06% of original model params). Because p-tuning shares parameters between tasks during training, p-tuning on many very different tasks at once might perform worse than prompt tuning, which tunes a distinct set of parameters for each task. For the same reason, p-tuning your model on multiple tasks that are similar might allow your model to share insight between tasks.


With all that covered, let's get started!


In [5]:
import os
import wget 
from omegaconf import OmegaConf

# Tasks and Datasets
We will be using p-tuning and prompt tuning to teach our GPT model to do 3 tasks: **Sentiment Analysis**, **Question Answering** and **Intent and Slot Classification**.

We will use [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for our sentiment analysis task, [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for question answering task, and []() for intent and slot classification. 

